In [2]:
import numpy as np 
import pandas as pd 
import matplotlib as plt 
import seaborn as sns 
import tensorflow as tf
from tensorflow import keras
import tensorflow.python.platform.build_info as build
from database_connection import DBConn

In [3]:
# check gpu availability 
print(f"TF is cuda build: {build.build_info['is_cuda_build']}")
print(f"Device list: {tf.config.list_physical_devices('GPU')}")

TF is cuda build: True
Device list: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# setup database connection 
HOST = 'localhost'
USER = 'db_user'
PORT = 5438
DATABASE = 'shot_db'
PASSWORD = 'LetMeIn'
TABLE_NAME = 'shot_data_table'

db = DBConn()

In [9]:
# grab a sample of the data to check out 
# value = db.query(f"SELECT * FROM {TABLE_NAME} WHERE shot_event<>'BLOCK' ORDER BY id ASC LIMIT 5000")
value = db.query(f"SELECT * FROM {TABLE_NAME} WHERE id = 531514")
value.columns 

Index(['id', 'game_id', 'season', 'game_date', 'game_period', 'shot_event',
       'seconds_elapsed', 'strength', 'strength_2', 'event_team', 'away_team',
       'home_team', 'position', 'shoots', 'player1', 'player2', 'player3',
       'away_player1', 'away_player2', 'away_player3', 'away_player4',
       'away_player5', 'away_player6', 'home_player1', 'home_player2',
       'home_player3', 'home_player4', 'home_player5', 'home_player6',
       'away_players', 'home_players', 'away_score', 'home_score',
       'away_goalie', 'home_goalie', 'home_coach', 'away_coach', 'event_zone',
       'x', 'y', 'is_home', 'goalie', 'catches', 'shot_type', 'loc', 'corsi',
       'fenwick', 'shot', 'goal', 'empty_net', 'xg', 'event_distance',
       'event_angle'],
      dtype='object')

In [10]:
value

,id,game_id,season,game_date,game_period,shot_event,seconds_elapsed,strength,strength_2,event_team,...,shot_type,loc,corsi,fenwick,shot,goal,empty_net,xg,event_distance,event_angle
0,531514,2010021026,20102011,2011-03-12,3,BLOCK,60,5x5,EV,PHI,...,None,0,1,0,0,0,0,0.0,21.0,0.0


In [11]:
value[['id', 'home_team', 'away_team', 'event_team', 'strength', 'strength_2', 'shot_event', 'shot_type', 'event_zone', 'x', 'y', 'event_distance', 'event_angle']]

,id,home_team,away_team,event_team,strength,strength_2,shot_event,shot_type,event_zone,x,y,event_distance,event_angle
0,531514,PHI,ATL,PHI,5x5,EV,BLOCK,None,Off,68,0,21.0,0.0


In [15]:
value.value_counts(value['shot_type'])

shot_type
Wrist          1824
Slap           1377
Snap            981
Backhand        371
Tip-In          340
Wrap-around      73
Deflected        34
Name: count, dtype: int64

In [8]:
type(value['shot_type'][0])

str